## Neural Networks

To obtain higher classification accuracy, we implemented neural networks, which we did not learn in the class. Multi-layer perceptrons neural network is a supervised method, and is very powerful in classifying Alzheimer's disease as shown below.

In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [2]:
# load data
df_train = pd.read_csv("data/ADNIMERGE_train.csv")
df_test = pd.read_csv("data/ADNIMERGE_test.csv")
X_train = df_train.drop(['RID', 'DX_bl'], axis=1).copy()
y_train = df_train['DX_bl'].copy()
X_test = df_test.drop(['RID', 'DX_bl'], axis=1).copy()
y_test = df_test['DX_bl'].copy()

In [3]:
# function to help compare the accuracy of models
def score(model, X_train, y_train, X_test, y_test):
    train_acc = model.score(X_train,y_train)
    test_acc = model.score(X_test,y_test)
    test_class0 = model.score(X_test[y_test==0], y_test[y_test==0])
    test_class1 = model.score(X_test[y_test==1], y_test[y_test==1])
    test_class2 = model.score(X_test[y_test==2], y_test[y_test==2])
    return pd.Series([train_acc, test_acc, test_class0, test_class1, test_class2],
                    index = ['Train accuracy', 'Test accuracy', 
                             "Test accuracy CN", "Test accuracy CI", "Test accuracy AD"])

In [4]:
cols_standardize = [
    c for c in X_train.columns 
    if (not c.startswith('PT')) or (c=='PTEDUCAT')]

X_train_std = X_train.copy()
X_test_std = X_test.copy()
for c in cols_standardize:
    col_mean = np.mean(X_train[c])
    col_sd = np.std(X_train[c])
    if col_sd > (1e-10)*col_mean:
        X_train_std[c] = (X_train[c]-col_mean)/col_sd
        X_test_std[c] = (X_test[c]-col_mean)/col_sd

In [5]:
print(X_train_std.shape)
X_train_std.head()

(621, 74)


,PTGENDER,PTEDUCAT,PTRACCAT_Asian,PTRACCAT_Black,PTRACCAT_Hawaiian/Other_PI,PTRACCAT_More_than_one,PTRACCAT_Unknown,PTRACCAT_White,PTETHCAT_Not_Hisp/Latino,PTMARRY_Married,...,WholeBrain,WholeBrain_slope,Entorhinal,Entorhinal_slope,Fusiform,Fusiform_slope,MidTemp,MidTemp_slope,ICV,ICV_slope
0,0,-2.852257,0,0,0,0,0,1,1,0,...,-1.761500,-0.567555,-0.820814,-1.269796,-1.426968,0.156847,-2.102069,-0.192827,-1.574482,0.093937
1,1,1.376909,0,0,0,0,0,1,1,1,...,-0.134464,-0.028641,-0.070387,0.188014,0.721399,-0.067438,0.019784,0.506511,-0.489132,-0.265646
2,0,0.607970,0,0,0,0,0,1,1,1,...,-1.300396,0.310720,0.456478,-0.560840,0.292776,0.016824,-0.650452,0.224140,-1.239633,-0.014198
3,0,-0.160970,0,0,0,0,0,1,1,0,...,-0.000094,-0.003749,0.006635,-0.003683,0.010325,0.015345,0.018697,0.004091,-0.005136,0.004314
4,1,-0.160970,0,0,0,0,0,1,1,0,...,-0.000094,-0.003749,0.006635,-0.003683,0.010325,0.015345,0.018697,0.004091,1.652198,-0.047345


In [6]:
# find the best parameters
cv_fold = KFold(n_splits=4, shuffle=True, random_state=9001)
parameters = {'alpha': [1e-3, 1e-2, 1e-1, 1, 1e1, 1e2],
              'hidden_layer_sizes': [(10), (25), (50), (100),
                                     (10, 10), (10, 25), (10, 50), 
                                     (25, 10), (25, 25), 
                                     (50, 10)]}
mlp = MLPClassifier(solver='lbfgs', activation='logistic', random_state=9001)
mlp_cv = GridSearchCV(mlp, parameters, cv=cv_fold)
mlp_cv.fit(X_train_std, y_train)
best_score = np.argmax(mlp_cv.cv_results_['mean_test_score'])
result = mlp_cv.cv_results_['params'][best_score]
a = result['alpha']
hidden_layer = result['hidden_layer_sizes']
mlp = MLPClassifier(solver='lbfgs', activation='logistic', random_state=9001,
                    alpha = a, hidden_layer_sizes=hidden_layer)
mlp = mlp.fit(X_train_std, y_train)

In [7]:
print("Optimal parameters")
print("L2 penalty parameter: ", a)
print("Hidden Layer Sizes: ", hidden_layer)
print('\n-----------------\n')
print("Training accuracy: ", mlp.score(X_train_std, y_train))
print("Test accuracy: ", mlp.score(X_test_std, y_test))
print('\n-----------------\n')
print('Test Confusion Matrix: ')
print(confusion_matrix(y_test, mlp.predict(X_test_std)))
nn_score = score(mlp, X_train_std, y_train, X_test_std, y_test)

Optimal parameters
L2 penalty parameter:  10.0
Hidden Layer Sizes:  25

-----------------

Training accuracy:  0.824476650564
Test accuracy:  0.777777777778

-----------------

Test Confusion Matrix: 
[[24 18  0]
 [ 8 79  6]
 [ 0  4 23]]


In [8]:
# random forest to compare with
rf_best = RandomForestClassifier(n_estimators=16, max_depth=8, random_state=9001)
rf_best.fit(X_train, y_train)
rf_score = score(rf_best, X_train, y_train, X_test, y_test)
print('\n-----------------\n')
print("Training accuracy: ", rf_best.score(X_train, y_train))
print("Test accuracy: ", rf_best.score(X_test, y_test))
print('\n-----------------\n')
print('Test Confusion Matrix: ')
print(confusion_matrix(y_test, rf_best.predict(X_test)))


-----------------

Training accuracy:  0.972624798712
Test accuracy:  0.796296296296

-----------------

Test Confusion Matrix: 
[[20 22  0]
 [ 4 87  2]
 [ 0  5 22]]


In [9]:
score_df = pd.DataFrame({"Neural Network": nn_score,
                         "Random Forest": rf_score})
score_df

,Neural Network,Random Forest
Train accuracy,0.824477,0.972625
Test accuracy,0.777778,0.796296
Test accuracy CN,0.571429,0.476190
Test accuracy CI,0.849462,0.935484
Test accuracy AD,0.851852,0.814815


The optimal hidden layer size is one hidden layer with 25 neurons. We need a l2-regularization term with value 10 to achieve the best accuracy.

The overall test accuracy of Neural Network is close to that of the best random forest in the previous model comparison section. However, we found that the test accuracy for the cognitive normal group and the Alzheimer's disease group is considerably higher using neural network. These two groups are what we are interested in.

We would prefer neural network model to the random forest because of its high classification accuracy of Alzheimer's disease.